In [1]:
import pandas as pd

df = pd.read_csv('C:\DataScience\AppliedAI\Kaggle\Titanic\\titanic_old.csv')
print(df.head(2))


   pclass  survived                           name     sex      age  sibsp  \
0       1         1   Allen Miss. Elisabeth Walton  female  29.0000      0   
1       1         1  Allison Master. Hudson Trevor    male   0.9167      1   

   parch  ticket      fare    cabin embarked boat  body  \
0      0   24160  211.3375       B5        S    2   NaN   
1      2  113781  151.5500  C22 C26        S   11   NaN   

                       home_dest  
0                    St Louis MO  
1  Montreal PQ / Chesterville ON  


In [2]:
df.dtypes

pclass         int64
survived       int64
name          object
sex           object
age          float64
sibsp          int64
parch          int64
ticket        object
fare         float64
cabin         object
embarked      object
boat          object
body         float64
home_dest     object
dtype: object

In [3]:
df["sex"] = df["sex"].astype('category')
df["embarked"] = df["embarked"].astype('category')
df.dtypes

pclass          int64
survived        int64
name           object
sex          category
age           float64
sibsp           int64
parch           int64
ticket         object
fare          float64
cabin          object
embarked     category
boat           object
body          float64
home_dest      object
dtype: object

In [4]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
df["sex_code"] = lb_make.fit_transform(df["sex"])
df[["sex", "sex_code"]].head(3)



,sex,sex_code
0,female,0
1,male,1
2,female,0


In [5]:
df["embarked_code"] = lb_make.fit_transform(df["embarked"])
df[["embarked", "embarked_code"]].head(3)

,embarked,embarked_code
0,S,2
1,S,2
2,S,2


In [6]:
null_columns=df.columns[df.isnull().any()]
df[null_columns].isnull().sum()

age           263
fare            1
cabin        1014
boat          823
body         1188
home_dest     564
dtype: int64

In [7]:
age_mean=df["age"].mean()
df["age"].fillna(age_mean, inplace = True) 

In [8]:
fare_mean=df["fare"].mean()
df["fare"].fillna(fare_mean, inplace = True) 


In [9]:
dff=df[["pclass","survived","age","sex_code","sibsp","parch","fare","embarked_code"]]
dff.head()

,pclass,survived,age,sex_code,sibsp,parch,fare,embarked_code
0,1,1,29.0000,0,0,0,211.3375,2
1,1,1,0.9167,1,1,2,151.5500,2
2,1,0,2.0000,0,1,2,151.5500,2
3,1,0,30.0000,1,1,2,151.5500,2
4,1,0,25.0000,0,1,2,151.5500,2


In [10]:
null_columns=dff.columns[dff.isnull().any()]
dff[null_columns].isnull().sum()

Series([], dtype: float64)

In [11]:
y = dff['survived']
X = dff.drop(['survived'],axis = 1)

print(X.head())
print("*"*75)
print(y.head())

   pclass      age  sex_code  sibsp  parch      fare  embarked_code
0       1  29.0000         0      0      0  211.3375              2
1       1   0.9167         1      1      2  151.5500              2
2       1   2.0000         0      1      2  151.5500              2
3       1  30.0000         1      1      2  151.5500              2
4       1  25.0000         0      1      2  151.5500              2
***************************************************************************
0    1
1    1
2    0
3    0
4    0
Name: survived, dtype: int64


In [12]:
dff.head()

,pclass,survived,age,sex_code,sibsp,parch,fare,embarked_code
0,1,1,29.0000,0,0,0,211.3375,2
1,1,1,0.9167,1,1,2,151.5500,2
2,1,0,2.0000,0,1,2,151.5500,2
3,1,0,30.0000,1,1,2,151.5500,2
4,1,0,25.0000,0,1,2,151.5500,2


In [33]:
# Pearson correlation coefficient
corr = dff.corr()["survived"].sort_values(ascending=False)[1:]
# absolute for positive values
abs_corr = abs(corr)

# random threshold for features to keep
relevant_features = abs_corr[abs_corr>0.2]
relevant_features.sort_values(ascending=False)[0:]

# now simply select the features needed 
new_df = df[relevant_features.index]

new_df.head()

,fare,pclass,sex_code
0,211.3375,1,0
1,151.5500,1,1
2,151.5500,1,0
3,151.5500,1,1
4,151.5500,1,0


In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)


(916, 7)
(393, 7)
(916,)
(393,)


In [17]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

C:\Users\naray\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [18]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(C=0.1);
from sklearn import tree
#clf = tree.DecisionTreeClassifier()
from sklearn.ensemble import RandomForestClassifier
#clf = RandomForestClassifier(max_depth=5, random_state=0)
from sklearn import ensemble
#clf = ensemble.GradientBoostingClassifier(loss='exponential', learning_rate=0.1, n_estimators=10)
    
clf.fit(X_train, Y_train);

preds=clf.predict(X_test)

C:\Users\naray\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [221]:


#w = clf.coef_

#import numpy as np
#print(np.count_nonzero(w))

from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
confusion_matrix(Y_test,preds)

array([[211,  35],
       [ 53,  94]], dtype=int64)

In [222]:
print(classification_report(Y_test,preds))
print("train accuracy:",clf.score(X_train,Y_train))
print("Test accuracy:",accuracy_score(Y_test, preds))

              precision    recall  f1-score   support

           0       0.80      0.86      0.83       246
           1       0.73      0.64      0.68       147

   micro avg       0.78      0.78      0.78       393
   macro avg       0.76      0.75      0.75       393
weighted avg       0.77      0.78      0.77       393

train accuracy: 0.7882096069868996
Test accuracy: 0.7760814249363868


In [223]:
from sklearn.model_selection import GridSearchCV
clf = LogisticRegression()
grid_values = {'penalty': ['l1', 'l2'],'C':[0.001,.009,0.01,.09,0.1,1,5,10,25]}
grid_clf_acc = GridSearchCV(clf, param_grid = grid_values,scoring = 'recall')
grid_clf_acc.fit(X_train, Y_train)

C:\Users\naray\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\naray\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\naray\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\naray\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\naray\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default s

C:\Users\naray\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\naray\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\naray\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\naray\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\naray\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'penalty': ['l1', 'l2'], 'C': [0.001, 0.009, 0.01, 0.09, 0.1, 1, 5, 10, 25]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='recall', verbose=0)

In [224]:
print("Best parameters set found on development set:")
print()
print(grid_clf_acc.best_params_)

Best parameters set found on development set:

{'C': 5, 'penalty': 'l2'}


In [225]:
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
#Predict values based on new parameters
y_pred_acc = grid_clf_acc.predict(X_test)

# New Model Evaluation metrics 
print('Accuracy Score : ' + str(accuracy_score(Y_test,y_pred_acc)))
print('Precision Score : ' + str(precision_score(Y_test,y_pred_acc)))
print('Recall Score : ' + str(recall_score(Y_test,y_pred_acc)))
print('F1 Score : ' + str(f1_score(Y_test,y_pred_acc)))

#Logistic Regression (Grid Search) Confusion matrix
confusion_matrix(Y_test,y_pred_acc)

Accuracy Score : 0.7938931297709924
Precision Score : 0.746268656716418
Recall Score : 0.6802721088435374
F1 Score : 0.7117437722419929


array([[212,  34],
       [ 47, 100]], dtype=int64)

In [230]:
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import StackingClassifier
import numpy as np
import warnings

warnings.simplefilter('ignore')

#clf1 = KNeighborsClassifier(n_neighbors=1)
clf2 = RandomForestClassifier(random_state=1)
clf3 = GaussianNB()
clf4 = XGBClassifier()
lr = LogisticRegression()
sclf = StackingClassifier(classifiers=[clf1, clf2, clf3, clf4], 
                          meta_classifier=lr)

print('3-fold cross validation:\n')

for clf, label in zip([clf2, clf3, clf4, sclf], 
                      ['Random Forest', 
                       'Naive Bayes',
                       'XGB',
                       'StackingClassifier']):

    scores = model_selection.cross_val_score(clf, X_train, Y_train, 
                                              cv=3, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" 
          % (scores.mean(), scores.std(), label))


3-fold cross validation:

Accuracy: 0.78 (+/- 0.02) [Random Forest]
Accuracy: 0.78 (+/- 0.02) [Naive Bayes]
Accuracy: 0.82 (+/- 0.01) [XGB]
Accuracy: 0.76 (+/- 0.01) [StackingClassifier]


In [12]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
model = XGBClassifier()
model.fit(X_train, Y_train)
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(Y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 79.90%


In [13]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

In [30]:
estimator = XGBClassifier(
    objective= 'binary:logistic',
    seed=42
)

In [31]:
#parameters = {
#    'max_depth': range (2, 10, 1),
#    'n_estimators': range(60, 220, 40),
#    'learning_rate': [0.1, 0.01, 0.05]
#}
parameters = { "learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
 "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
 "min_child_weight" : [ 1, 3, 5, 7 ],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ] }

In [32]:
grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters,
    scoring = 'accuracy',
    cv = 3,
    n_jobs=8,
    verbose=True
)

In [33]:
from datetime import datetime
start = datetime. now()
grid_search.fit(X_train, Y_train)
print("Time taken =", start-datetime. now())

Fitting 3 folds for each of 3840 candidates, totalling 11520 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    5.3s
[Parallel(n_jobs=4)]: Done 341 tasks      | elapsed:   12.5s
[Parallel(n_jobs=4)]: Done 841 tasks      | elapsed:   24.5s
[Parallel(n_jobs=4)]: Done 1541 tasks      | elapsed:   47.3s
[Parallel(n_jobs=4)]: Done 2441 tasks      | elapsed:  1.2min
[Parallel(n_jobs=4)]: Done 3541 tasks      | elapsed:  1.6min
[Parallel(n_jobs=4)]: Done 4841 tasks      | elapsed:  2.2min
[Parallel(n_jobs=4)]: Done 6341 tasks      | elapsed:  2.8min
[Parallel(n_jobs=4)]: Done 8041 tasks      | elapsed:  3.7min
[Parallel(n_jobs=4)]: Done 9941 tasks      | elapsed:  4.8min
[Parallel(n_jobs=4)]: Done 11513 out of 11520 | elapsed:  5.7min remaining:    0.1s
[Parallel(n_jobs=4)]: Done 11520 out of 11520 | elapsed:  5.7min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42, silent=True,
       subsample=1),
       fit_params=None, iid='warn', n_jobs=4,
       param_grid={'learning_rate': [0.05, 0.1, 0.15, 0.2, 0.25, 0.3], 'max_depth': [3, 4, 5, 6, 8, 10, 12, 15], 'min_child_weight': [1, 3, 5, 7], 'gamma': [0.0, 0.1, 0.2, 0.3, 0.4], 'colsample_bytree': [0.3, 0.4, 0.5, 0.7]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=True)

In [35]:
grid_search.best_params_

{'colsample_bytree': 0.5,
 'gamma': 0.4,
 'learning_rate': 0.05,
 'max_depth': 6,
 'min_child_weight': 1}

In [36]:
y_pred = grid_search.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(Y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 80.15%


In [39]:
model = XGBClassifier(
    objective= 'binary:logistic',
    seed=42,
    colsample_bytree=0.5,
    gamma=0.4,
    learning_rate=0.05,
    max_depth=6,
    min_child_weight=1
)

In [40]:
model.fit(X_train, Y_train)
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(Y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 80.15%
